**Troubleshooting Scripts - Detecting Inefficient Queries with xEvents**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Capturing Expensive Queries In event\_file Target**

Capturing queries that consume more than 5,000ms of CPU time or issued more than 50,000 logical reads or writes. Change filters based on your workload

Tune events and actions based on what you are trying to accomplish. Do not collect information you dont need.

Change file location in event\_file target

In [ ]:
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'Expensive Queries')
	DROP EVENT SESSION [Expensive Queries] ON SERVER;
GO

CREATE EVENT SESSION [Expensive Queries] 
ON SERVER
ADD EVENT sqlserver.sql_statement_completed
(
    ACTION
    (
        sqlserver.client_app_name
        ,sqlserver.client_hostname
        ,sqlserver.database_id
        ,sqlserver.plan_handle
        ,sqlserver.query_hash
        ,sqlserver.query_plan_hash
        ,sqlserver.sql_text
        ,sqlserver.username
    )    
    WHERE
    (
        (
            cpu_time >= 5000000 or -- Time in microseconds
            logical_reads >= 50000 or
            writes >= 50000
        ) AND 
        sqlserver.is_system = 0 
    ) 
)
,ADD EVENT sqlserver.sp_statement_completed
(
    ACTION
    (
        sqlserver.client_app_name
        ,sqlserver.client_hostname
        ,sqlserver.database_id
        ,sqlserver.plan_handle
        ,sqlserver.query_hash
        ,sqlserver.query_plan_hash
        ,sqlserver.sql_text
        ,sqlserver.username
    )    
    WHERE
    (
        (
            cpu_time >= 5000000 or -- Time in microseconds
            logical_reads >= 50000 or
            writes >= 50000
        ) AND 
        sqlserver.is_system = 0 
    ) 
)
ADD TARGET package0.event_file
(
    SET FILENAME = 'C:\ExtEvents\Expensive Queries.xel'
)
WITH
(
    event_retention_mode=allow_single_event_loss
    ,max_dispatch_latency=30 seconds
);

Start the session.

It adds the overhead. Do not keep it running permanently

In [ ]:
ALTER EVENT SESSION [Expensive Queries] ON SERVER STATE = START;

Stop the session

In [ ]:
ALTER EVENT SESSION [Expensive Queries] ON SERVER STATE = STOP;

**SQL Server 2017+.** Parse the results. This is resource intensive and can take some time with large amount of data.

Do not parse unnecesary information

In [ ]:
IF OBJECT_ID(N'tempdb..#EventData') IS NOT NULL
	DROP TABLE #EventData;

IF OBJECT_ID(N'tempdb..#Queries') IS NOT NULL
	DROP TABLE #Queries;
GO

CREATE TABLE #EventData
(
  event_data XML NOT NULL,
  file_name NVARCHAR(260) NOT NULL,
  file_offset BIGINT NOT NULL,
  timestamp_utc datetime2(7) NOT NULL -- SQL Server 2017+
);

INSERT INTO #EventData(event_data, file_name, file_offset, timestamp_utc)
  SELECT CONVERT(XML,event_data), file_name, file_offset, timestamp_utc 
  FROM sys.fn_xe_file_target_read_file
    ('c:\extevents\Expensive Queries*.xel',NULL,NULL,NULL);

;WITH EventInfo([Event],[Event Time],[DB],[Statement],[SQL],[User Name]
  ,[Client],[App],[CPU Time],[Duration],[Logical Reads]
  ,[Physical Reads],[Writes],[Rows],[Query Hash],[Plan Hash]
  ,[PlanHandle],[Stmt Offset],[Stmt Offset End],File_Name,File_Offset)
AS
(
  SELECT
    event_data.value('/event[1]/@name','SYSNAME') AS [Event]
    ,timestamp_utc AS [Event Time] -- SQL Server 2017+
    /*,event_data.value('/event[1]/@timestamp','DATETIME') 
        AS [Event Time] -- Prior SQL Server 2017 */
    ,event_data.value('((/event[1]/action[@name="database_id"]/value/text())[1])'
         ,'INT') AS [DB]
    ,event_data.value('((/event[1]/data[@name="statement"]/value/text())[1])'
         ,'NVARCHAR(MAX)') AS [Statement]
    ,event_data.value('((/event[1]/action[@name="sql_text"]/value/text())[1])'
         ,'NVARCHAR(MAX)') AS [SQL]
    ,event_data.value('((/event[1]/action[@name="username"]/value/text())[1])'
         ,'NVARCHAR(255)') AS [User Name]
    ,event_data.value('((/event[1]/action[@name="client_hostname"]/value/text())[1])'
         ,'NVARCHAR(255)') AS [Client]
    ,event_data.value('((/event[1]/action[@name="client_app_name"]/value/text())[1])'
         ,'NVARCHAR(255)') AS [App]
    ,event_data.value('((/event[1]/data[@name="cpu_time"]/value/text())[1])'
         ,'BIGINT') AS [CPU Time]
    ,event_data.value('((/event[1]/data[@name="duration"]/value/text())[1])'
         ,'BIGINT') AS [Duration]
    ,event_data.value('((/event[1]/data[@name="logical_reads"]/value/text())[1])'
         ,'INT') AS [Logical Reads]
    ,event_data.value('((/event[1]/data[@name="physical_reads"]/value/text())[1])'
         ,'INT') AS [Physical Reads]
    ,event_data.value('((/event[1]/data[@name="writes"]/value/text())[1])'
         ,'INT') AS [Writes]
    ,event_data.value('((/event[1]/data[@name="row_count"]/value/text())[1])'
         ,'INT') AS [Rows]
    ,event_data.value('xs:hexBinary(((/event[1]/action[@name="query_hash"]/value/text())[1]))'
         ,'BINARY(8)') AS [Query Hash]
    ,event_data.value('xs:hexBinary(((/event[1]/action[@name="query_plan_hash"]/value/text())[1]))'
         ,'BINARY(8)') AS [Plan Hash]
    ,event_data.value('xs:hexBinary(((/event[1]/action[@name="plan_handle"]/value/text())[1]))'
         ,'VARBINARY(64)') AS [PlanHandle]
    ,event_data.value('((/event[1]/data[@name="offset"]/value/text())[1])'
         ,'INT') AS [Stmt Offset]    
    ,event_data.value('((/event[1]/data[@name="offset_end"]/value/text())[1])'
         ,'INT') AS [Stmt Offset End]
    ,file_name
    ,file_offset
  FROM 
    #EventData 
)
SELECT 
  ei.*
  ,TRY_CONVERT(XML,qp.Query_Plan) AS [Plan]
INTO #Queries
FROM 
  EventInfo ei 
    OUTER APPLY 
      sys.dm_exec_text_query_plan
      (
        ei.PlanHandle
        ,ei.[Stmt Offset]
        ,ei.[Stmt Offset End]
      ) qp
OPTION (MAXDOP 1, RECOMPILE);

SELECT TOP 100 * FROM #Queries;


**SQL Server 2012 - 2016.** Parse the results. This is resource intensive and can take some time with large amount of data.

Do not parse unnecesary information

In [ ]:
IF OBJECT_ID(N'tempdb..#EventData') IS NOT NULL
	DROP TABLE #EventData;

IF OBJECT_ID(N'tempdb..#Queries') IS NOT NULL
	DROP TABLE #Queries;
GO

CREATE TABLE #EventData
(
  event_data XML NOT NULL,
  file_name NVARCHAR(260) NOT NULL,
  file_offset BIGINT NOT NULL
);

INSERT INTO #EventData(event_data, file_name, file_offset)
  SELECT CONVERT(XML,event_data), file_name, file_offset 
  FROM sys.fn_xe_file_target_read_file
    ('c:\extevents\Expensive Queries*.xel',NULL,NULL,NULL);

;WITH EventInfo([Event],[Event Time],[DB],[Statement],[SQL],[User Name]
  ,[Client],[App],[CPU Time],[Duration],[Logical Reads]
  ,[Physical Reads],[Writes],[Rows],[Query Hash],[Plan Hash]
  ,[PlanHandle],[Stmt Offset],[Stmt Offset End],File_Name,File_Offset)
AS
(
  SELECT
    event_data.value('/event[1]/@name','SYSNAME') AS [Event]
    ,event_data.value('/event[1]/@timestamp','DATETIME') 
        AS [Event Time] 
    ,event_data.value('((/event[1]/action[@name="database_id"]/value/text())[1])'
         ,'INT') AS [DB]
    ,event_data.value('((/event[1]/data[@name="statement"]/value/text())[1])'
         ,'NVARCHAR(MAX)') AS [Statement]
    ,event_data.value('((/event[1]/action[@name="sql_text"]/value/text())[1])'
         ,'NVARCHAR(MAX)') AS [SQL]
    ,event_data.value('((/event[1]/action[@name="username"]/value/text())[1])'
         ,'NVARCHAR(255)') AS [User Name]
    ,event_data.value('((/event[1]/action[@name="client_hostname"]/value/text())[1])'
         ,'NVARCHAR(255)') AS [Client]
    ,event_data.value('((/event[1]/action[@name="client_app_name"]/value/text())[1])'
         ,'NVARCHAR(255)') AS [App]
    ,event_data.value('((/event[1]/data[@name="cpu_time"]/value/text())[1])'
         ,'BIGINT') AS [CPU Time]
    ,event_data.value('((/event[1]/data[@name="duration"]/value/text())[1])'
         ,'BIGINT') AS [Duration]
    ,event_data.value('((/event[1]/data[@name="logical_reads"]/value/text())[1])'
         ,'INT') AS [Logical Reads]
    ,event_data.value('((/event[1]/data[@name="physical_reads"]/value/text())[1])'
         ,'INT') AS [Physical Reads]
    ,event_data.value('((/event[1]/data[@name="writes"]/value/text())[1])'
         ,'INT') AS [Writes]
    ,event_data.value('((/event[1]/data[@name="row_count"]/value/text())[1])'
         ,'INT') AS [Rows]
    ,event_data.value('xs:hexBinary(((/event[1]/action[@name="query_hash"]/value/text())[1]))'
         ,'BINARY(8)') AS [Query Hash]
    ,event_data.value('xs:hexBinary(((/event[1]/action[@name="query_plan_hash"]/value/text())[1]))'
         ,'BINARY(8)') AS [Plan Hash]
    ,event_data.value('xs:hexBinary(((/event[1]/action[@name="plan_handle"]/value/text())[1]))'
         ,'VARBINARY(64)') AS [PlanHandle]
    ,event_data.value('((/event[1]/data[@name="offset"]/value/text())[1])'
         ,'INT') AS [Stmt Offset]    
    ,event_data.value('((/event[1]/data[@name="offset_end"]/value/text())[1])'
         ,'INT') AS [Stmt Offset End]
    ,file_name
    ,file_offset
  FROM 
    #EventData 
)
SELECT 
  ei.*
  ,TRY_CONVERT(XML,qp.Query_Plan) AS [Plan]
INTO #Queries
FROM 
  EventInfo ei 
    OUTER APPLY 
      sys.dm_exec_text_query_plan
      (
        ei.PlanHandle
        ,ei.[Stmt Offset]
        ,ei.[Stmt Offset End]
      ) qp
OPTION (MAXDOP 1, RECOMPILE);

SELECT TOP 100 * FROM #Queries;

**Capturing Expensive Queries In ring\_buffer Target**  

Capturing queries that consume more than 1,000ms of CPU time or issued more than 10,000 logical reads or writes. Change filters based on your workload

Tune events and actions based on what you are trying to accomplish. Do not collect information you dont need.

This session is using ring\_buffer target and can be used for quick profiling (should not run more than a couple minutes)

In [ ]:
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'Query Profiling')
	DROP EVENT SESSION [Query Profiling] ON SERVER;
GO

CREATE EVENT SESSION [Query Profiling] 
ON SERVER
ADD EVENT
    sqlserver.sql_statement_completed
    (
        ACTION
        (
            sqlserver.client_app_name
            ,sqlserver.client_hostname
            ,sqlserver.database_id
            ,sqlserver.plan_handle
			,sqlserver.query_hash
			,sqlserver.query_plan_hash
            ,sqlserver.sql_text
            ,sqlserver.username
        )    
        WHERE
        (
            (
                cpu_time >= 1000000 or -- Time in microseconds
                logical_reads >= 10000 or
                writes >= 10000
            ) AND 
            sqlserver.is_system = 0 
        ) 
    )
,ADD EVENT
	sqlserver.sp_statement_completed
    (
        ACTION
        (
            sqlserver.client_app_name
            ,sqlserver.client_hostname
            ,sqlserver.database_id
            ,sqlserver.plan_handle
			,sqlserver.query_hash
			,sqlserver.query_plan_hash
            ,sqlserver.sql_text
            ,sqlserver.username
        )    
        WHERE
        (
            (
                cpu_time >= 1000000 or -- Time in microseconds
                logical_reads >= 10000 or
                writes >= 10000
            ) AND 
            sqlserver.is_system = 0 
        ) 
    )
ADD TARGET package0.ring_buffer
(SET max_events_limit=0, max_memory=4096)
WITH
    (
        event_retention_mode=allow_single_event_loss
        ,max_dispatch_latency=30 seconds
    );

Start the session.

It adds the overhead. Do not keep it running permanently.

You need to keep the session active while parsing the results

In [ ]:
ALTER EVENT SESSION [Query Profiling] ON SERVER STATE = START;

Check how many events were captured and lost due to sys.xe\_session\_target function limitations

In [ ]:
DECLARE 
    @TargetData XML;

SELECT  
    @TargetData = CONVERT(XML,st.target_data) 
FROM  
    sys.dm_xe_sessions s WITH (NOLOCK)  
        JOIN sys.dm_xe_session_targets st WITH(NOLOCK) ON 
            s.address = st.event_session_address 
WHERE  
    s.name = 'Query Profiling' and st.target_name = 'ring_buffer';

;WITH Cnt(target_count, node_count)
AS
(
	SELECT 
		@TargetData.value('(RingBufferTarget/@eventCount)[1]','INT') 
        ,@TargetData.value('count(RingBufferTarget/event)', 'INT')
)
SELECT
    target_count
	,node_count
	,target_count - node_count AS [lost]
FROM
	Cnt;

Parsing the results

In [ ]:
IF OBJECT_ID(N'tempdb..#Events') IS NOT NULL
	DROP TABLE #Events;

IF OBJECT_ID(N'tempdb..#Queries') IS NOT NULL
	DROP TABLE #Queries;
GO

CREATE TABLE #Events
(
    [Event Time] DATETIME2(7) NOT NULL,
    event_data XML NOT NULL
);

DECLARE 
    @TargetData XML;

SELECT  
    @TargetData = CONVERT(XML,st.target_data) 
FROM  
    sys.dm_xe_sessions s WITH (NOLOCK)  
        JOIN sys.dm_xe_session_targets st WITH(NOLOCK) ON 
            s.address = st.event_session_address 
WHERE  
    s.name = 'Query Profiling' AND st.target_name = 'ring_buffer';

INSERT INTO #Events([Event Time], event_data)
    SELECT 
        t.e.value('@timestamp','DATETIME2(7)'), t.e.query('.') 
    FROM 
        @TargetData.nodes('/RingBufferTarget/event') AS t(e); 

;WITH EventInfo([Event],[Event Time],[DB],[Statement],[SQL],[User Name]
    ,[Client],[App],[CPU Time],[Duration],[Logical Reads]
    ,[Physical Reads],[Writes],[Rows],[Query Hash],[Plan Hash]
	,[PlanHandle],[Stmt Offset],[Stmt Offset End])
AS
(
  SELECT
	event_data.value('/event[1]/@name','SYSNAME') AS [Event]
	,[Event Time] 
    ,event_data.value('((/event[1]/action[@name="database_id"]/value/text())[1])'
	    ,'INT') AS [DB]
    ,event_data.value('((/event[1]/data[@name="statement"]/value/text())[1])'
        ,'NVARCHAR(MAX)') AS [Statement]
    ,event_data.value('((/event[1]/action[@name="sql_text"]/value/text())[1])'
        ,'NVARCHAR(MAX)') AS [SQL]
    ,event_data.value('((/event[1]/action[@name="username"]/value/text())[1])'
        ,'NVARCHAR(255)') AS [User Name]
    ,event_data.value('((/event[1]/action[@name="client_hostname"]/value/text())[1])'
        ,'NVARCHAR(255)') AS [Client]
    ,event_data.value('((/event[1]/action[@name="client_app_name"]/value/text())[1])'
        ,'NVARCHAR(255)') AS [App]
    ,event_data.value('((/event[1]/data[@name="cpu_time"]/value/text())[1])'
        ,'BIGINT') AS [CPU Time]
    ,event_data.value('((/event[1]/data[@name="duration"]/value/text())[1])'
        ,'BIGINT') AS [Duration]
    ,event_data.value('((/event[1]/data[@name="logical_reads"]/value/text())[1])'
        ,'INT') AS [Logical Reads]
    ,event_data.value('((/event[1]/data[@name="physical_reads"]/value/text())[1])'
        ,'INT') AS [Physical Reads]
    ,event_data.value('((/event[1]/data[@name="writes"]/value/text())[1])'
        ,'INT') AS [Writes]
    ,event_data.value('((/event[1]/data[@name="row_count"]/value/text())[1])'
        ,'INT') AS [Rows]
    ,event_data.value(
        'xs:hexBinary(((/event[1]/action[@name="query_hash"]/value/text())[1]))'
            ,'BINARY(8)') AS [Query Hash]
    ,event_data.value(
        'xs:hexBinary(((/event[1]/action[@name="query_plan_hash"]/value/text())[1]))'
            ,'BINARY(8)') AS [Plan Hash]
    ,event_data.value(
        'xs:hexBinary(((/event[1]/action[@name="plan_handle"]/value/text())[1]))'
            ,'VARBINARY(64)') AS [PlanHandle]
    ,event_data.value('((/event[1]/data[@name="offset"]/value/text())[1])'
	    ,'INT') AS [Stmt Offset]    
	,event_data.value('((/event[1]/data[@name="offset_end"]/value/text())[1])'
	    ,'INT') AS [Stmt Offset End]
  FROM 
    #Events
)
SELECT 
  ei.*
  ,TRY_CONVERT(XML,qp.Query_Plan) AS [Plan]
INTO 
	#Queries
FROM 
    EventInfo ei 
		OUTER APPLY 
		    sys.dm_exec_text_query_plan
            (
                ei.PlanHandle
                ,ei.[Stmt Offset]
                ,ei.[Stmt Offset End]
            ) qp
OPTION (MAXDOP 1, RECOMPILE);

SELECT TOP 100 * FROM #Queries;


Stop the session

In [ ]:
ALTER EVENT SESSION [Query Profiling] ON SERVER STATE = STOP;